In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='14cbc94c-3b09-49fa-a27b-ec51ddfb8526', project_access_token='p-cfbf735a530330b181785334a493efa1271867e5')
pc = project.project_context

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('matplotlib', matplotlib.__version__)
print('seaborn', sns.__version__)

numpy 1.15.4
pandas 0.24.1
matplotlib 3.0.2
seaborn 0.9.0


# Importing Kaggle data

In [3]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_62d16fde9ee64a49b809102844695402 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XSCASHjDajDxaWQPvUeAapDDHRz6wRS_Hd2vgjN-IDMf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_62d16fde9ee64a49b809102844695402.get_object(Bucket='predictingtheenergyoutputofwindtu-donotdelete-pr-4zv0a0xlcw57fc',Key='T1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


# Importing Weather dataset

In [4]:
# Downloading the package
!pip install wwo-hist

In [5]:
# keys of the weather API
keys = [""]

In [6]:
len(keys)

142

In [7]:
# importing function for retriving historical data 
from wwo_hist import retrieve_hist_data

In [8]:
# libraries for date time
from datetime import datetime, date, timedelta
import time

In [11]:
# frequency means no_of_hours  
frequency=1

new_data = []
# count to keep track of no_of_records collected
count = 0

# current_key_count to check the no_of_calls of current_key
current_key_count = 0

# key_track to keep track of key
key_track = 0
current_key = keys[key_track]

# max number of requests from single key in one day
max_key_count = 500

# requests_time_out to keep track of number of times the request timed out 
requests_time_out = 0

# The latitude and longitude of wind farm of dataset in turkey, yalova
lat_long = ["40.6549,29.2842"]
for i in data.values:
    end_date = datetime.strptime(i[0], '%d %m %Y %H:%M')
    start_date = end_date
    one_row = []

    if current_key_count > max_key_count:
        key_track += 1
        current_key = keys[key_track]
        current_key_count = 0
    try:
        hist_weather_data = retrieve_hist_data(current_key,
                                  lat_long,
                                  start_date,
                                  end_date,
                                  frequency,
                                  location_label = False,
                                  export_csv = False,
                                  store_df = True
                                  )
        current_key_count += 1
        records = hist_weather_data[0]
# converting columns to numeric type 
        records = records[["maxtempC", "WindGustKmph", "DewPointC", "WindChillC", "humidity", "precipMM", "pressure"]].apply(pd.to_numeric)
        maxtempC = records["maxtempC"].values
        windGustKmph = records["WindGustKmph"].values
        DewPointC = records["DewPointC"].values
        windChillC = records["WindChillC"].values
        humidity = records["humidity"].values
        precipMM = records["precipMM"].values
        pressure = records["pressure"].values         
        
#taking the average of every weather feature as data coming is from one hour before the date_time requested
        one_row.append([ end_date,  sum(maxtempC) / len(maxtempC), sum(windGustKmph) / len(windGustKmph), 
                         sum(DewPointC) / len(DewPointC), sum(windChillC) / len(windChillC),
                         sum(humidity) / len(humidity), sum(precipMM) / len(precipMM), sum(pressure) / len(pressure)])

    except HTTPError:
        requests_time_out += 1

    new_data.extend(one_row)  

    count += 1

    if count % 1000 == 0:
        print("Count:{}, Len(Final data):{}, key track:{}".format(count, len(new_data), key_track))



Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


NameError: name 'HTTPError' is not defined

In [13]:
len(new_data)

50530


In [14]:
data_integrated = np.concatenate( (data.values, np.array(new_data)), axis = 1 )

In [15]:
data = pd.DataFrame( data_integrated, columns = ["DateTime", "LV ActivePower", "Wind Speed (m/s)",
                                               "Theoretical_Power_Curve (KWh)", "Wind Direction","end_date",  "maxtempC",
                                               "windGustKmph","DewPointC", "windChillC", "humidity", "precipMM", "pressure"])


In [16]:
from project_lib import Project
project = Project(pc,'<ProjectID>','<ProjectAccessToken>')

project.save_data(file_name = "Dataset.csv",data = df.to_csv(index=False))